In [1]:
library(dplyr)
library(tibble)
dataDir <- "~/labdata/dataShare_from_Mingfeng/10xgenome/Nicola_monkey_10X/genexp/"


Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
rpkm <- read.table(paste0(dataDir, "nicola.monkeyNESdiff.gene.RPKM.txt"), header = TRUE, stringsAsFactors = FALSE) %>%
        column_to_rownames("Geneid") %>%
        as.matrix()
count <- read.table(paste0(dataDir, "nicola.monkeyNESdiff.gene.count.txt"), header = TRUE, stringsAsFactors = FALSE) %>%
        column_to_rownames("Geneid") %>%
        as.matrix()

In [3]:
head(rpkm)

,RM_1F_77,RM_4F_77,RM_4C_77,RM_4GE_77,RM_5F_77,RM_5C_77,RM_5GE_77,RM_6F_77,RM_6C_77,RM_6GE_77,⋯,RM_8F_42,RM_8C_42,RM_8GE_42,RM_9F_42,RM_9C_42,RM_9GE_42,RM_2GE_77,RM_3F_77,RM_3C_77,RM_3GE_77
PGBD2,2.0069,2.8310,2.8306,2.9694,2.8602,2.6911,2.6750,4.1757,3.3134,3.8668,⋯,3.8758,4.0875,4.6206,4.2411,4.5914,4.8440,3.0234,2.2727,2.8602,2.7812
LOC114674980,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
TRNAE-CUC,4.8155,6.1456,1.8640,3.6460,4.5153,3.4293,3.5628,2.7903,1.5257,3.7361,⋯,0.7393,0.6893,2.0353,2.4996,0.9396,1.8092,4.0565,2.8806,2.0812,0.9757
ZNF692,6.9547,9.9274,10.5333,9.6553,8.9711,10.2942,10.7591,8.3759,7.3474,8.1489,⋯,6.1464,7.7852,9.7491,7.1344,9.1413,7.7205,8.6851,7.2286,9.3372,9.1973
TRNAL-CAA,1.5479,0.0000,0.0000,0.0000,0.8601,0.0000,0.0000,0.9567,1.3077,0.5337,⋯,0.2112,1.1816,1.0467,0.3571,1.6108,0.0000,0.0000,0.9876,0.4460,0.0000
LOC114678402,0.6426,1.0406,0.8462,0.8053,0.8310,0.7574,0.7432,0.6573,0.9173,0.9168,⋯,0.4535,0.7358,0.4945,0.5367,0.4150,0.3996,0.9386,0.9543,0.8044,0.9877


In [4]:
## Update rownames to match the seurat modification
rownames(rpkm) <- gsub("_", "-", rownames(rpkm))
rownames(count) <- gsub("_", "-", rownames(count))


## sample meta data
meta <- data.frame(row.names = colnames(rpkm),
                samplename = colnames(rpkm),
                stringsAsFactors = FALSE)
meta$age <- strsplit(meta$samplename, "_", fixed = TRUE) %>%
                sapply(., function(x) x[3]) %>%
                paste0("E", .)
meta$dayregion <- strsplit(meta$samplename, "_", fixed = TRUE) %>%
                sapply(., function(x) x[2])
meta$days <- sapply(meta$dayregion, function(x) gsub("[A-Z]", "", x)) %>%
                as.numeric()
meta$region <- sapply(meta$dayregion, function(x) gsub("[0-9]", "", x)) %>%
                gsub("^F$", "FC", .) %>%
                gsub("^C$", "OC", .)
head(meta)
save(count, rpkm, meta, file = paste0("./load_files/NSC_invitro_bulk_RNA.Rdata"))

,samplename,age,dayregion,days,region
,<chr>,<chr>,<chr>,<dbl>,<chr>
RM_1F_77,RM_1F_77,E77,1F,1,FC
RM_4F_77,RM_4F_77,E77,4F,4,FC
RM_4C_77,RM_4C_77,E77,4C,4,OC
RM_4GE_77,RM_4GE_77,E77,4GE,4,GE
RM_5F_77,RM_5F_77,E77,5F,5,FC
RM_5C_77,RM_5C_77,E77,5C,5,OC


In [5]:
## Remove GE

submeta <- meta[meta$region != "GE", ]
subcount <- count[, rownames(submeta)]
subrpkm <- rpkm[, rownames(submeta)]
save(subcount, subrpkm, submeta, file = paste0("./load_files/NSC_invitro_dorsal_bulk_RNA.Rdata"))


In [6]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux Server 7.9 (Maipo)

Matrix products: default
BLAS:   /gpfs/ycga/home/sm2726/packages/R_versions/R-3.6.1/lib64/R/lib/libRblas.so
LAPACK: /gpfs/ycga/home/sm2726/packages/R_versions/R-3.6.1/lib64/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tibble_3.0.1 dplyr_1.0.7 

loaded via a namespace (and not attached):
 [1] magrittr_2.0.3    tidyselect_1.1.0  uuid_1.1-0        R6_2.4.1         
 [5] rlang_0.4.11      fansi_0.4.1      